### Import Libraries

In [1]:
# ===== Auto Reload =====
%load_ext autoreload
%autoreload 2

# ===== Standard Libraries =====
import numpy as np
import matplotlib.pyplot as plt
import scipy
import random
import os
import re


# ===== PyTorch =====
import torch
import torch.nn as nn
from torchdiffeq import odeint

# torchdiffeq 버전 확인
import torchdiffeq
print(f"torchdiffeq version: {torchdiffeq.__version__}")
print("")

# ===== Check Installation =====
print("="*50)
print("Environment Check")
print("="*50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print(f"CUDA available: {torch.cuda.is_available()}")
print("="*50)
print("")


# ===== Your Custom Module =====
# Import from battery_benchmark_wrapper
import sys
from pathlib import Path

project_root = Path(r"C:\Users\ljw76\Documents\Python\PyTorch_NeuralODE")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from neuralODE.utils import *
from battery_benchmark_wrapper import*

print("="*50)
print("✓ All modules loaded!")
print("="*50)


torchdiffeq version: 0.2.5

Environment Check
PyTorch version: 2.5.1
CUDA device: NVIDIA RTX 3500 Ada Generation Laptop GPU
CUDA available: True

✓ All modules loaded!


### Data Import

In [2]:
# MATLAB 데이터 디렉토리 경로
data_dir = r"C:\Users\ljw76\Documents\MATLAB\LFP_SAFT\data\learning_data2"

# ===== 온도 필터 설정 =====
target_temps = [25]  # 원하는 온도 리스트 (예: [25, 35] 또는 [] 이면 모든 온도)

# 디렉토리 내 모든 .mat 파일 목록 가져오기
mat_files = [f for f in os.listdir(data_dir) if f.endswith('.mat')]
print(f"Found {len(mat_files)} .mat files:")
for file in mat_files:
    print(f"  - {file}")

# 모든 파일 로드
print(f"\nLoading files with temperature filter: {target_temps if target_temps else 'All'}")
all_loaded_data = {}

for i, file in enumerate(mat_files):
    file_path = os.path.join(data_dir, file)
    print(f"Loading {i+1}/{len(mat_files)}: {file}")
    
    try:
        mat_data = scipy.io.loadmat(file_path)
        
        # 온도 추출 (파일명에서, 대문자 C만 찾기)
        temp_match = re.search(r'_(\d+)C_', file)
        file_temp = int(temp_match.group(1)) if temp_match else None
        
        # 온도 필터링
        if target_temps and (file_temp is None or file_temp not in target_temps):
            print(f"  ⚠ Skipped (temp: {file_temp}°C)")
            continue
        
        # 파일명에서 확장자 제거하여 키로 사용
        file_key = file.replace('.mat', '')
        
        # 메타데이터 키 제외한 실제 데이터만 저장
        data_keys = [key for key in mat_data.keys() if not key.startswith('__')]
        all_loaded_data[file_key] = {}
        
        for key in data_keys:
            all_loaded_data[file_key][key] = mat_data[key]
        
        print(f"  ✓ Loaded (temp: {file_temp}°C) - {len(data_keys)} data keys: {data_keys}")
        
    except Exception as e:
        print(f"  ✗ Failed to load {file}: {e}")

print(f"\nAll files loaded!")
print(f"Successfully loaded {len(all_loaded_data)} files:")
for file_key, data_dict in all_loaded_data.items():
    print(f"  {file_key}: {list(data_dict.keys())}")

# 첫 번째 파일의 데이터 구조 상세 확인
if all_loaded_data:
    first_file_key = list(all_loaded_data.keys())[0]
    first_data_key = list(all_loaded_data[first_file_key].keys())[0]
    sample_data = all_loaded_data[first_file_key][first_data_key]

    print(f"\nSample data structure from {first_file_key}[{first_data_key}]:")
    print(f"  Shape: {sample_data.shape}")
    print(f"  Type: {type(sample_data)}")
    if hasattr(sample_data, 'dtype'):
        print(f"  Dtype: {sample_data.dtype}")
        if hasattr(sample_data.dtype, 'names') and sample_data.dtype.names:
            print(f"  Field names: {sample_data.dtype.names}")



Found 152 .mat files:
  - gitt_restonly_25C_SOC_10.mat
  - gitt_restonly_25C_SOC_100.mat
  - gitt_restonly_25C_SOC_11.mat
  - gitt_restonly_25C_SOC_12.mat
  - gitt_restonly_25C_SOC_13.mat
  - gitt_restonly_25C_SOC_14.mat
  - gitt_restonly_25C_SOC_15.mat
  - gitt_restonly_25C_SOC_16.mat
  - gitt_restonly_25C_SOC_17.mat
  - gitt_restonly_25C_SOC_18.mat
  - gitt_restonly_25C_SOC_19.mat
  - gitt_restonly_25C_SOC_2.mat
  - gitt_restonly_25C_SOC_20.mat
  - gitt_restonly_25C_SOC_21.mat
  - gitt_restonly_25C_SOC_22.mat
  - gitt_restonly_25C_SOC_23.mat
  - gitt_restonly_25C_SOC_25.mat
  - gitt_restonly_25C_SOC_26.mat
  - gitt_restonly_25C_SOC_27.mat
  - gitt_restonly_25C_SOC_28.mat
  - gitt_restonly_25C_SOC_29.mat
  - gitt_restonly_25C_SOC_3.mat
  - gitt_restonly_25C_SOC_30.mat
  - gitt_restonly_25C_SOC_31.mat
  - gitt_restonly_25C_SOC_32.mat
  - gitt_restonly_25C_SOC_33.mat
  - gitt_restonly_25C_SOC_34.mat
  - gitt_restonly_25C_SOC_35.mat
  - gitt_restonly_25C_SOC_36.mat
  - gitt_restonly_25C_

### Convert Struct to Data Frame

In [3]:
# ===== Key 선택 =====
target_keys = ["time", "Vref", "Vspme", "current", "temperature", "c_s_n_bulk", "soc_n", "ocp"]

# ===== 모든 파일 처리 =====
extracted_data = {}  # Dictionary로 저장

excluded_target = []
excluded_data = {}
# extracted_data는 excluded_target에 들어있지 않은 것만, excluded_data는 target에 들어있는 것만 저장하도록 아래 for문 수정

print("="*60)
print("Converting all structs to DataFrames")
print("="*60)

for file_key, data_dict in all_loaded_data.items():
    print(f"\nProcessing: {file_key}")
    
    # struct 추출 (첫 번째 key)
    struct_key = list(data_dict.keys())[0]
    struct_data = data_dict[struct_key]
    
    # DataFrame 변환
    df = struct2df(struct_data, selected_keys=target_keys)

    # 저장: excluded_target 리스트에 있으면 excluded_data에, 아니면 extracted_data에
    if file_key in excluded_target:
        excluded_data[file_key] = df
        print(f"  ✓ DataFrame created (EXCLUDED): {df.shape}")
    else:
        extracted_data[file_key] = df
        print(f"  ✓ DataFrame created: {df.shape}")
    print(f"  Columns: {list(df.columns)}")

print("\n" + "="*60)
print(f"✓ Total {len(extracted_data)} DataFrames created!")
print(f"✓ Total {len(excluded_data)} EXCLUDED DataFrames created!")
print("="*60)

# 요약
for name, df in extracted_data.items():
    print(f"  {name}: {df.shape}")

for name, df in excluded_data.items():
    print(f"  (EXCLUDED) {name}: {df.shape}")
# extracted_data는 excluded_target에 들어있지 않은 것만 들어가게끔 for문에서 처리 (아래 for문 참고!)



Converting all structs to DataFrames

Processing: gitt_restonly_25C_SOC_10
Available keys in MATLAB struct:
   1. time
   2. current
   3. temperature
   4. Vout
   5. Ve
   6. Vcond
   7. eta_p
   8. eta_n
   9. Un
  10. Up
  11. ocp
  12. soc_n
  13. soc_p
  14. c_s_p_surf
  15. c_s_n_surf
  16. c_s_p_bulk
  17. c_s_n_bulk
  18. i0p
  19. k_cs
  20. alpha_cs
  21. tau_cs
  22. c_s_p_surf_tilde
  23. Dsn_eff
  24. Dsp_eff
  25. Vspme
  26. Vref
Total: 26 keys

Selected keys:
  1. time ✓
  2. Vref ✓
  3. Vspme ✓
  4. current ✓
  5. temperature ✓
  6. c_s_n_bulk ✓
  7. soc_n ✓
  8. ocp ✓

Extracting data:
------------------------------------------------------------
  ✓ time                : shape (3601, 1)       → 3601 points
  ✓ Vref                : shape (3601, 1)       → 3601 points
  ✓ Vspme               : shape (3601, 1)       → 3601 points
  ✓ current             : shape (3601, 1)       → 3601 points
  ✓ temperature         : shape (3601, 1)       → 3601 points
  ✓ c_s_n_bulk   

### Data Pre-Processing

In [4]:
# 'soc_n' 값이 0.96 ~ 0.12 (96% ~ 12%) 사이인 데이터만 target_data에 저장
# target_data를 리스트(list)로 저장 (DataFrame만 담김, key는 저장하지 않음)
drivingonly_keywords = ['drivingonly']
norest_keywords = ['norest']


################## Driving Only Training ##################
drivingonly_list = extract_data(extracted_data, drivingonly_keywords, soc_lb=0.0, soc_ub=1.0)
norest_list_temp = extract_data(extracted_data, norest_keywords, soc_lb=0.0, soc_ub=1.0)

drivingonly_list = remove_duplicates(drivingonly_list)

norest_list_temp = remove_duplicates(norest_list_temp)
norest_list = norest_list_temp[:6] + norest_list_temp[7:]
norest_final_test = [norest_list_temp[6]]

norest_list_split = split_df(norest_list, window_minutes=20, time_col='time', random_seed=42)

drivingonly_dict_list = df2dict(drivingonly_list)
norest_dict_list = df2dict(norest_list_split)
norest_dict_final_test = df2dict(norest_final_test)


norest_dict_training, norest_dict_test = split_train_test(norest_dict_list, 0.8)
drivingonly_dict_training, drivingonly_dict_test = split_train_test(drivingonly_dict_list, 0.8)


driving_dict_training = drivingonly_dict_training + norest_dict_training
driving_dict_test = drivingonly_dict_test + norest_dict_test + norest_dict_final_test



################## Rest Only Training ##################
rest_keywords = ['restonly']

rest_list = extract_data(extracted_data, rest_keywords, soc_lb=0.0, soc_ub=1.0)

rest_list = remove_duplicates(rest_list)

rest_list_smooth = smooth_Vcorr(rest_list, 51) # de-noise data after downsampling

rest_list_split = split_df(rest_list_smooth, window_minutes=20, time_col='time', random_seed=42)

rest_dict_list = df2dict(rest_list_split)

rest_dict_training, rest_dict_test = split_train_test(rest_dict_list, 0.8)




################## Joint Training ##################
combined_dict_training = drivingonly_dict_training + norest_dict_training + rest_dict_training
combined_dict_test = drivingonly_dict_test + norest_dict_test + norest_dict_final_test + rest_dict_test


Key: gitt_restonly_25C_SOC_10, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_100, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_11, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_12, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_13, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_14, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_15, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_16, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_17, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_18, keyword_match: False, type: <class 'pandas.core.frame.DataFrame'>
Key: gitt_restonly_25C_SOC_19

### DNN

In [ ]:
from pathlib import Path
import sys
import torch
from torch.utils.data import DataLoader
from collections import deque
import copy

project_root = Path(r"C:\Users\ljw76\Documents\Python\PyTorch_NeuralODE")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from Benchmarks.battery_benchmark_wrapper import DNNBenchmark, build_dnn_dataset

# Dataset (teacher forcing: k -> k+1)
dnn_dataset = build_dnn_dataset(combined_dict_training)  # 필요 시 combined_dict_training으로 교체
dnn_loader = DataLoader(dnn_dataset, batch_size=256, shuffle=True, drop_last=True)

# Model / optimizer / loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dnn_model = DNNBenchmark().to(device)
dnn_optimizer = torch.optim.AdamW(dnn_model.parameters(), lr=1e-3, eps=1e-8, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    dnn_optimizer,
    mode="min",
    factor=0.5,
    patience=10,
    min_lr=1e-10,
    threshold=1e-3,
    threshold_mode="rel",
)
criterion = torch.nn.MSELoss()

num_epochs = 1000
best_rmse = float("inf")
best_epoch = -1
best_lr = dnn_optimizer.param_groups[0]["lr"]
best_state_dict = None
rmse_hist = deque(maxlen=20)

for epoch in range(num_epochs):
    dnn_model.train()
    running_loss = 0.0
    voltage_sse = 0.0
    sample_count = 0
    grad_norm_before = 0.0
    grad_norm_after = 0.0

    for xb, yb, v_spme_next, v_meas_next in dnn_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        v_spme_next = v_spme_next.to(device)
        v_meas_next = v_meas_next.to(device)

        preds = dnn_model(xb)
        loss = criterion(preds, yb)
        dnn_optimizer.zero_grad()
        loss.backward()

        grad_before = 0.0
        for p in dnn_model.parameters():
            if p.grad is not None:
                grad_before += p.grad.data.norm(2).item() ** 2
        grad_norm_before += grad_before ** 0.5

        torch.nn.utils.clip_grad_norm_(dnn_model.parameters(), max_norm=50.0)

        grad_after = 0.0
        for p in dnn_model.parameters():
            if p.grad is not None:
                grad_after += p.grad.data.norm(2).item() ** 2
        grad_norm_after += grad_after ** 0.5

        dnn_optimizer.step()

        running_loss += loss.item() * xb.size(0)
        voltage_sse += torch.sum((v_spme_next + preds.detach() - v_meas_next) ** 2).item()
        sample_count += xb.size(0)

    epoch_loss = running_loss / len(dnn_loader.dataset)
    epoch_rmse = (voltage_sse / sample_count) ** 0.5
    scheduler.step(epoch_loss)
    avg_grad_before = grad_norm_before / len(dnn_loader)
    avg_grad_after = grad_norm_after / len(dnn_loader)

    rmse_hist.append(epoch_rmse * 1e3)

    if epoch_rmse < best_rmse:
        best_rmse = epoch_rmse
        best_epoch = epoch + 1
        best_lr = dnn_optimizer.param_groups[0]["lr"]
        best_state_dict = copy.deepcopy(dnn_model.state_dict())
        print(f"[DNN] ✅ Best RMSE improved to {best_rmse*1e3:.2f} mV at epoch {best_epoch}")

    if len(rmse_hist) == rmse_hist.maxlen and (max(rmse_hist) - min(rmse_hist)) <= 0.005:
        print("[DNN] Window range <= 0.005 mV → early stop")
        break

    print(
        f"[DNN] epoch {epoch+1}/{num_epochs}, "
        f"loss={epoch_loss:.3e}, "
        f"RMSE={epoch_rmse*1e3:.2f} mV, "
        f"LR={dnn_optimizer.param_groups[0]['lr']:.3e}, "
        f"Grad={avg_grad_before:.3e}->{avg_grad_after:.3e}"
    )

print(f"[DNN] Best RMSE = {best_rmse*1e3:.2f} mV at epoch {best_epoch}, best LR={best_lr:.3e}")
if best_state_dict is not None:
    torch.save(best_state_dict, project_root / "Benchmarks" / "dnn_best_state_dict.pth")
    print("[DNN] Saved best state dict.")

[DNN] ✅ Best RMSE improved to 2.73 mV at epoch 1
[DNN] epoch 1/3000, loss=7.428e-06, RMSE=2.73 mV, LR=1.000e-03, Grad=1.239e-03->1.239e-03
[DNN] ✅ Best RMSE improved to 1.64 mV at epoch 2
[DNN] epoch 2/3000, loss=2.688e-06, RMSE=1.64 mV, LR=1.000e-03, Grad=9.697e-04->9.697e-04
[DNN] ✅ Best RMSE improved to 1.60 mV at epoch 3
[DNN] epoch 3/3000, loss=2.571e-06, RMSE=1.60 mV, LR=1.000e-03, Grad=8.187e-04->8.187e-04
[DNN] ✅ Best RMSE improved to 1.58 mV at epoch 4
[DNN] epoch 4/3000, loss=2.495e-06, RMSE=1.58 mV, LR=1.000e-03, Grad=7.764e-04->7.764e-04


KeyboardInterrupt: 